# Parameter Fitting for Empirical Flow Stress Models

In [ ]:
# only for Google Colab
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/requirements.txt"
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/flow-stress/bst.csv"
!pip install -r "requirements.txt"

Import of required packages.

In [ ]:
import pandas as pd  # for data loading
import numpy as np  # for vectorized computations
import scipy.optimize as opt  # for least squares fitting
import plotly.express as px  # for plotting

## Loading the Experimental Data

We have the flow stress data in a long format CSV file and load it via `pandas` into a data frame.

In [ ]:
data = pd.read_csv(
    "bst.csv",  # file name
    encoding="utf8",  # use unicode to be safe on Windows systems (default on UNIX)
    sep=",",  # columns separated by comma
)
data

As the data is in long format (columns for temperature, strain and strain rate; rows for every data point), it is very comfortable to plot it with `plotly`.
But first we sort by temperature, then by strain rate and last by strain.

In [ ]:
data.sort_values(
    by=["temp", "rate", "strain"],  # multi-level sorting columns
    ascending=True,  # order from small to large
    inplace=True,  # use the existing data frame, do not copy the data
)
data

Now we can plot the data in dependence of the test conditions.

In [ ]:
px.line(
    data,
    "strain",  # strain on the x-axis
    "stress",  # stress on the y-axis
    color="temp",  # distinguish temperatures by colors
    facet_col="rate",  # draw multiple plots for distinct strain rates
    line_group="file",  # distinguish lines by file name, avoid zick-zack curves
)

## Defining the Model

We use the Freiberg flow stress model to fit on the data as given below.
We define a python function that computes this equation using `numpy` (shorthand `np`) to be able to efficiently iterate over all data points.
The free parameters for fitting are stored in the vector `params` (`params[i]`=$m_i$, `params[0]`=$A$).

$$ k_\mathrm{f} \left( \varphi, \dot{\varphi}, \vartheta \right) = A \cdot \exp\left(m_1 \vartheta\right)\cdot \vartheta^{m_9}
\cdot \varphi^{m_2} \cdot \exp\left( \frac{m_4}{\varphi} \right)
\cdot \left(1 + \varphi \right)^{m_5\vartheta+m_6} \cdot \exp\left(m_7 \varphi\right)
\cdot \dot{\varphi}^{m_3 + m_8 \vartheta} $$

In [ ]:
def freiberg_model(strain, rate, temp, params):
    strain = (
        strain + 0.01
    )  # some small strain value to avoid singularities near strain=0
    rate = rate + 0.01  # respectively
    return (
        params[0]
        * np.exp(params[1] * temp)
        * temp ** params[9]
        * strain ** params[2]
        * np.exp(params[4] / strain)
        * (1 + strain) ** (params[5] * temp + params[6])
        * np.exp(params[7] * strain)
        * rate ** (params[3] + params[8] * temp)
    )

## Fit the Model to the Data

Now we use the leat squares method to find an optimal fit of the model to the data (implemented in `scipy.optimize.least_squares`).
We define a function that computes the absolute error between the data and the model at every data point for a given set of model parameters (`params`).
Then, we pass it to the optimization routine which returns the optimal fit to us.

In [ ]:
def model_data_error(params):
    return freiberg_model(data.strain, data.rate, data.temp, params) - data.stress


fit = opt.least_squares(
    model_data_error,
    x0=[
        1e3,
        -1e-2,
        1e-1,
        1e-1,
        -1e-1,
        1e-4,
        1e-1,
        1e-1,
        1e-4,
        0.1,
    ],  # initial guess of the params vector
)
fit

## Plot the Model Predictions Counter the Data

First we create a fine cartesian raster to evaluate the model on, so we get smooth curves of the model prediction.

In [ ]:
strains = np.linspace(0, 1.5, 50)  # strain with 50 points between 0 and 1.2
temps = (
    data.temp.unique()
)  # take only the distinct temperatures that are present in the data
rates = data.rate.unique()  # respectively

grid = pd.MultiIndex.from_product(
    [temps, rates, strains], names=["temp", "rate", "strain"]
)
grid

The raster is an index for a data frame, but we want it as an actual dataframe for easier computation.

In [ ]:
model_data = pd.DataFrame(index=grid).reset_index()
model_data

Then we apply the model function with our determined best parameters and save the results in an additonal column in the data frame.

In [ ]:
model_data["stress"] = freiberg_model(
    model_data.strain, model_data.rate, model_data.temp, fit.x
)
model_data

We can plot the results as before with the data.

In [ ]:
px.line(model_data, "strain", "stress", color="temp", facet_col="rate")

To plot both in comparison, we first have to merge the data frames. We distinguish model predictions and experimental data by an additonal column containg a marker label.

In [ ]:
combined_data = pd.concat(
    [model_data, data],  # list of the frames to combine
    keys=["model", "exp"],  # list of marker labels in same order as above
    names=["type", "index"],  # names of the marker column and the index column
)
combined_data.reset_index(
    level=0, inplace=True
)  # make the type column a normal column (was an index column)
combined_data.fillna(
    value={"file": ""}, inplace=True
)  # fill missing file in model results
combined_data

Now we plot the data as before, but distinguish the origin by lime style (solid and dashed).

In [ ]:
px.line(
    combined_data,
    "strain",
    "stress",
    color="temp",
    facet_col="rate",
    line_dash="type",
    line_group="file",
)